## **Feature Processing**

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import LineString

# NUMBER OF DATA POINTS
NUM_DATA_POINTS = 10000

print('Reading Shapefile...')

# Load the shapefile
shapefile_path = 'C:\\Users\\User\\Downloads\\6414_Test\\GTAA_errors\\GTAA_errors.shp'
gdf = gpd.read_file(shapefile_path)
print('Shapefile Loaded!\n')

# exploding
gdf = gdf.explode(index_parts=True).reset_index()

class DataProcessing:
    completedCount = 0

    def remove_duplicates(self, gdf):
        gdf_temp = gdf.iloc[:, 1:].copy()
        gdf_temp = gdf_temp.drop(columns=['Error'])
        dup_mask = gdf_temp.duplicated(keep='first')

        gdf_res = gdf[~dup_mask]
        gdf_res.loc[:, 'Error'] = gdf_res['Error'].apply(lambda x: 0 if x == 2 else x)
        return gdf_res

    def compute_linestring_metrics(self, linestring):
        """
        Computes multiple metrics for a LINESTRING Z:
        - is_closed: Whether the linestring forms a closed path.
        - length_3d: The total 3D length of the linestring.
        - line_curvature: The ratio of actual path length to straight-line distance.
        - num_vertices: The number of vertices in the linestring.
        - vertex_density: The number of vertices per unit length.
        - total_angle_change: The sum of absolute angle changes between segments.
        - is_point: Whether the linestring consists of a single repeated point.
        - num_connections: Number of other lines that touch this line, but don't go through (i.e. a 3-way intersection, not a 4-way intersection)
        - num_intersections: Number of other lines that pass through this line
        - bounding_box_width: The width of the bounding box of the linestring.
        - bounding_box_height: The height of the bounding box of the linestring.
        - relative_angle_change: Ratio of total_angle_change to length of the line. (Relative Angle Change to Line Length)
        - proximity_to_neighbors: The average minimum distance to neighboring lines.
        - similarity_score: Similarity score with close neighbor lines based on selected metrics (length, curvature, vertices) and normalized based on neighbor group
        - num_neighbors: Number of neighboring lines within a radius equal to 1% of the total map size.
        """
        if not isinstance(linestring, LineString) or len(linestring.coords) < 2:
            return False, 0.0, 1.0, 0, 0.0, 0.0, True, 0, 0 ,0.0, 0.0, 0.0, 0.0, 0.0, 0 # Defaults for invalid linestrings

        coords = np.array(linestring.coords)
        first_point = coords[0]
        last_point = coords[-1]
        is_closed = np.array_equal(first_point, last_point)

        diffs = np.diff(coords, axis=0)
        segment_lengths = np.linalg.norm(diffs, axis=1)
        sum_length = np.sum(segment_lengths)

        straight_line_distance = np.linalg.norm(last_point - first_point)
        line_curvature = sum_length / straight_line_distance if straight_line_distance > 0 else 1.0

        num_vertices = len(coords)

        # Vertices per unit length
        vertex_density = num_vertices / sum_length if sum_length > 0 else 0.0

        # Compute total angle change
        if len(linestring.coords) == 2:
            total_angle_change = 0.0
        else:
            valid_segments = segment_lengths > 0
            unit_vectors = diffs[valid_segments] / segment_lengths[:, None][valid_segments]
            dot_products = np.einsum("ij,ij->i", unit_vectors[:-1], unit_vectors[1:])
            angles = np.arccos(np.clip(dot_products, -1.0, 1.0))
            # Ensure we sum all angle changes without taking absolute values
            total_angle_change = np.sum(angles)

        # Check if all coordinates are the same (i.e., single repeated point)
        is_point = np.all(coords == coords[0])

        # Compute number of connections (lines that share at least one point)
        connected_lines_idx = list(spatial_index.intersection(linestring.bounds))
        connected_lines = gdf.iloc[connected_lines_idx]
        # Subtract 1 to exclude the line itself
        num_connections = sum(connected_lines.geometry.touches(linestring)) 

        # Compute number of lines that intersect with this line
        intersecting_lines_idx = list(
            spatial_index.intersection(linestring.bounds))
        intersecting_lines = gdf.iloc[intersecting_lines_idx]
        # Subtract 1 to exclude the line itself
        num_intersections = sum(intersecting_lines.geometry.intersects(linestring)) - 1

        # Compute bounding box width and height
        minx, miny, maxx, maxy = linestring.bounds
        bounding_box_width = maxx - minx
        bounding_box_height = maxy - miny

        # Get the bounding box of the current line
        buffered_line = linestring.buffer(search_radius)
        possible_neighbors = list(spatial_index.intersection(buffered_line.bounds))

        num_neighbors = len(possible_neighbors) - 1

        # Compute proximity to neighbors (average minimum distance)
        min_distances = []
        for neighbor_idx in possible_neighbors:
            neighbor = gdf.iloc[neighbor_idx].geometry
            if neighbor != linestring:  # Avoid self-comparison
                distance = linestring.distance(neighbor)
                min_distances.append(distance)

        # If there are neighbors, compute the average distance
        proximity_to_neighbors = search_radius * 2
        # Compute weighted proximity: Closer neighbors have higher weight
        if min_distances:
            weights = [1 / d if d > 0 else 1 for d in min_distances]
            proximity_to_neighbors = np.average(min_distances, weights=weights)
        else:
            proximity_to_neighbors = search_radius * 2  # Default if no neighbors

        # Compute similarity score with neighbors
        similarity_score = 0.0
        neighbor_metrics = []
        for neighbor_idx in possible_neighbors:
            neighbor = gdf.iloc[neighbor_idx].geometry
            if neighbor != linestring:  # Avoid self-comparison
                neighbor_coords = np.array(neighbor.coords)
                neighbor_diffs = np.diff(neighbor_coords, axis=0)
                neighbor_segment_lengths = np.linalg.norm(neighbor_diffs, axis=1)
                neighbor_sum_length = np.sum(neighbor_segment_lengths)

                neighbor_first_point = coords[0]
                neighbor_last_point = coords[-1]
                neighbor_straight_line_distance = np.linalg.norm(
                    neighbor_last_point - neighbor_first_point)
                neighbor_line_curvature = neighbor_sum_length / \
                    neighbor_straight_line_distance if neighbor_straight_line_distance > 0 else 1.0
                
                neighbor_num_vertices = len(neighbor_coords)

                neighbor_metrics.append(
                    [neighbor_sum_length, neighbor_line_curvature, neighbor_num_vertices])

        # Extract and normalize metrics for the neighbor group
        lengths = np.array([metrics[0] for metrics in neighbor_metrics])
        curvatures = np.array([metrics[1] for metrics in neighbor_metrics])
        vertices = np.array([metrics[2] for metrics in neighbor_metrics])

        # Normalize the current line's metrics against the neighbor group
        length_mean, length_std = np.mean(lengths), np.std(lengths)
        curvature_mean, curvature_std = np.mean(curvatures), np.std(curvatures)
        vertices_mean, vertices_std = np.mean(vertices), np.std(vertices)

        normalized_length = (sum_length - length_mean) / \
            length_std if length_std > 0 else 0.0
        normalized_curvature = (line_curvature - curvature_mean) / \
            curvature_std if curvature_std > 0 else 0.0
        normalized_vertices = (num_vertices - vertices_mean) / \
            vertices_std if vertices_std > 0 else 0.0

        # Calculate similarity score using normalized metrics
        if (num_neighbors > 0):
            similarity_score = 1 / (1 + abs(normalized_length) + abs(normalized_curvature) +
                                abs(normalized_vertices))

        relative_angle_change = total_angle_change / sum_length if sum_length != 0 else 0

        self.completedCount += 1
        if self.completedCount % 1000 == 0:
            print(f'Finished {self.completedCount} rows')

        return is_closed, sum_length, line_curvature, num_vertices, vertex_density, total_angle_change, is_point, num_connections, num_intersections, bounding_box_width, bounding_box_height, relative_angle_change, proximity_to_neighbors, similarity_score, num_neighbors

gdf = gdf.reset_index()
gdf['id'] = gdf['index']

processing = DataProcessing()
# Shuffle the GeoDataFrame and take the top 10,000 rows
# Shuffling the GeoDataFrame
gdf = processing.remove_duplicates(gdf)

gdf1 = gdf[gdf['Error'] == 1]
gdf2 = gdf[gdf['Error'] != 1]
gdf2 = gdf2.head(NUM_DATA_POINTS)  # Take the top 10,000 rows
gdf = gpd.GeoDataFrame(pd.concat([gdf1, gdf2], ignore_index=True))

gdf = gdf.sample(frac=1, random_state=42)
gdf = gdf.reset_index(drop=True)
spatial_index = gdf.sindex

print('Duplicates Removed + Random Sampling + Set Spacial Index\n')

minx, miny, maxx, maxy = gdf.total_bounds

# Compute the width and height of the bounding box
bbox_width = maxx - minx
bbox_height = maxy - miny
search_radius = 0.01 * max(bbox_width, bbox_height)

print('Computing Features...')
# Apply the function and ensure the output shape matches the expected columns
metrics = gdf.geometry.apply(processing.compute_linestring_metrics)
print('Features Computed!\n')

# Convert the result to a DataFrame and check the lengths
metrics_df = pd.DataFrame(metrics.tolist(), columns=[
                          "is_closed", "sum_length", "line_curvature", "num_vertices", "vertex_density", "total_angle_change", "is_point", "num_connections", "num_intersections", "bounding_box_width", "bounding_box_height", "relative_angle_change", "proximity_to_neighbors", "similarity_score", "num_neighbors"])

# Assign the calculated metrics to the original GeoDataFrame
gdf = gdf.join(metrics_df)

# Compute metrics for each line
columns = ["id", "is_closed", "sum_length", "line_curvature", "num_vertices", "vertex_density",
           "total_angle_change", "is_point", "num_connections", "num_intersections", "bounding_box_width", "bounding_box_height", "relative_angle_change", "proximity_to_neighbors", "similarity_score", "num_neighbors", "Error"]
gdf_features = gdf[columns]
gdf_features = gdf_features.reset_index()
print('Final Training Set Created with Labels')

output_path = '.\\GTAA_Errors_with_metrics.csv'
gdf_features.to_csv(output_path, index=False)
print('Training Set Stored')

gdf_features.head(30)

Reading Shapefile...
Shapefile Loaded!

Duplicates Removed + Random Sampling + Set Spacial Index

Computing Features...


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\_methods.py:227: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\_methods.py:184: RuntimeWarning: invalid value enco

Finished 1000 rows
Finished 2000 rows
Finished 3000 rows
Finished 4000 rows
Finished 5000 rows
Finished 6000 rows
Finished 7000 rows
Finished 8000 rows
Finished 9000 rows
Finished 10000 rows
Features Computed!

Final Training Set Created with Labels
Training Set Stored


,index,id,is_closed,sum_length,line_curvature,num_vertices,vertex_density,total_angle_change,is_point,num_connections,num_intersections,bounding_box_width,bounding_box_height,relative_angle_change,proximity_to_neighbors,similarity_score,num_neighbors,Error
0,0,156838,False,5.799891,2.707294,6,1.034502,5.269789,False,0,1,2.0073,2.1411,0.908601,10.992623,0.323633,300,1
1,1,6898,True,14.995678,1.000000,17,1.133660,23.564956,False,0,2,2.9789,2.9592,1.571450,4.710499,0.487796,23,0
2,2,6941,True,7.000716,1.000000,5,0.714213,4.712721,False,0,0,2.5874,2.3029,0.673177,2.689995,0.618077,61,0
3,3,509,False,0.901641,1.000000,2,2.218177,0.000000,False,0,0,0.6290,0.6460,0.000000,5.571404,0.362947,88,0
4,4,9176,False,0.680246,1.000000,2,2.940115,0.000000,False,0,0,0.1836,0.6550,0.000000,6.536263,0.475676,110,0
5,5,3585,False,0.666627,1.000000,2,3.000177,0.000000,False,0,0,0.6340,0.2060,0.000000,0.145352,0.487826,41,0
6,6,6368,False,28.613736,1.002612,7,0.244638,0.231117,False,1,1,22.0389,18.1321,0.008077,0.121865,0.636115,88,0
7,7,4795,False,23.633402,1.000000,2,0.084626,0.000000,False,0,1,0.4010,23.6300,0.000000,4.675020,0.585729,160,0
8,8,7668,False,0.680179,1.000000,2,2.940403,0.000000,False,0,0,0.4839,0.4780,0.000000,6.511975,0.514190,116,0
9,9,7399,False,1.299248,1.000000,2,1.539352,0.000000,False,0,0,0.9084,0.9289,0.000000,8.157686,0.370880,41,0


In [66]:
print(gdf[['geometry']])

                                                 geometry
0       LINESTRING Z (609028.044 4838381.721 0, 609028...
1       LINESTRING Z (609025.924 4838380.128 0, 609025...
2       LINESTRING Z (609024.331 4838381.717 0, 609024...
3       LINESTRING Z (609025.921 4838383.31 0, 609026....
4       LINESTRING Z (609028.044 4838381.721 0, 609028...
...                                                   ...
162399  LINESTRING Z (609757.796 4838065.622 0, 609750...
162400  LINESTRING Z (609056.916 4838089 0, 609138.352...
162401  LINESTRING Z (609064.663 4838129.67 0, 609062....
162402  LINESTRING Z (608991.717 4838072.881 0, 608992...
162403  LINESTRING Z (608999.066 4838065.382 0, 608999...

[162404 rows x 1 columns]


## [Obsolete] **DWG-CAD Features**

In [25]:
import ezdxf
from shapely.geometry import Polygon, LineString, Point
from shapely.ops import transform
from pyproj import CRS, Transformer


def extract_geometries_from_dwg(dwg_path):
    doc = ezdxf.readfile(dwg_path)
    msp = doc.modelspace()

    geometries = []
    for entity in msp:
        if entity.dxftype() == 'LWPOLYLINE' or entity.dxftype() == 'POLYLINE':
            points = [(p[0], p[1]) for p in entity.get_points()]
            geometry = Polygon(
                points) if entity.is_closed else LineString(points)
            geometries.append(geometry)
        elif entity.dxftype() == 'CIRCLE':
            center = entity.dxf.center
            radius = entity.dxf.radius
            geometry = Point(center).buffer(radius)
            geometries.append(geometry)
    return gpd.GeoDataFrame(geometry=geometries, crs="EPSG:4326")


def map_shapes(dwg_path, shp_path):
    # Extract geometries from DWG
    dwg_gdf = extract_geometries_from_dwg(dwg_path)

    # Load shapefile
    shp_gdf = gpd.read_file(shp_path)

    # Ensure CRS match
    if dwg_gdf.crs != shp_gdf.crs:
        shp_gdf = shp_gdf.to_crs(dwg_gdf.crs)

    # Create spatial indices for faster searching
    dwg_gdf['geometry'] = dwg_gdf['geometry'].buffer(0)
    shp_gdf['geometry'] = shp_gdf['geometry'].buffer(0)
    dwg_index = dwg_gdf.sindex
    shp_index = shp_gdf.sindex

    # Mapping using Intersection Over Union (IoU)
    shape_mapping = {}
    for dwg_idx, dwg_shape in dwg_gdf.iterrows():
        possible_matches_index = list(
            shp_index.intersection(dwg_shape.geometry.bounds))
        possible_matches = shp_gdf.iloc[possible_matches_index]

        best_match = None
        best_score = 0

        for shp_idx, shp_shape in possible_matches.iterrows():
            intersection_area = dwg_shape.geometry.intersection(
                shp_shape.geometry).area
            union_area = dwg_shape.geometry.union(shp_shape.geometry).area
            iou = intersection_area / union_area if union_area > 0 else 0

            if iou > best_score:
                best_score = iou
                best_match = shp_idx

        if best_match is not None:
            shape_mapping[dwg_idx] = best_match

    print("Shape Mapping Complete:", shape_mapping)
    return shape_mapping


# Example usage
cad_path = '.\\GTAA.dwg'
shp_path = 'C:\\Users\\User\\Downloads\\GTAA_SHAPEFILES\\GTAA_POLYGON.shp'
result = map_shapes(cad_path, shp_path)
print(result)

KeyboardInterrupt: 

In [ ]:

def map_shapes(cad_path, shp_path):
    # Load CAD and SHP files
    cad_gdf = gpd.read_file(cad_path)
    shp_gdf = gpd.read_file(shp_path)

    # Ensure the CRS matches
    if cad_gdf.crs != shp_gdf.crs:
        shp_gdf = shp_gdf.to_crs(cad_gdf.crs)

    # Create a mapping dictionary
    shape_mapping = {}

    # Iterate through CAD shapes to find corresponding SHP shapes
    for cad_index, cad_shape in cad_gdf.iterrows():
        best_match = None
        best_score = float('inf')

        for shp_index, shp_shape in shp_gdf.iterrows():
            # Compute Hausdorff distance or Intersection over Union (IoU)
            intersection_area = cad_shape.geometry.intersection(
                shp_shape.geometry).area
            union_area = cad_shape.geometry.union(shp_shape.geometry).area
            iou = intersection_area / union_area if union_area > 0 else 0

            # Check for the highest overlap using IoU
            if iou > 0.7 and iou < best_score:
                best_match = shp_index
                best_score = iou

        # Save the mapping
        if best_match is not None:
            shape_mapping[cad_index] = best_match

    return shape_mapping


# Example usage
cad_path = 'path_to_cad_file.dxf'
shp_path = 'path_to_shapefile.shp'
result = map_shapes(cad_path, shp_path)
print(result)